In [702]:
import pandas as pd
import numpy as np

# task_1
df = pd.read_csv(
    "horse_data.csv",
    usecols=[0, 1, 3, 4, 5, 6, 10, 22],
    names=[
        "surgery?",
        "Age",
        "rectal temperature",
        "pulse",
        "respiratory rate",
        "temperature of extremities",
        "pain",
        "outcome",
    ],
    header=0,
    na_values=np.nan,
)

df.head()

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1,1,39.2,88,20,?,3,3
1,2,1,38.30,40,24,1,3,1
2,1,9,39.10,164,84,4,2,2
3,2,1,37.30,104,35,?,?,2
4,2,1,?,?,?,2,2,1


In [703]:
# task_2

# работаем с категориальными величинами
df = df.apply(
    lambda x: pd.to_numeric(x, errors="coerce")
)  # меняем все ненумерованные значения на np.nan
df["Age"] = df["Age"].replace(
    {9: 2}
)  # логически можно сделать вывод, что вместо 9 имели ввиду 2. Заменяем


def validate_categorical_values(row, *args):
    if row in args:
        return row
    else:
        return np.nan


# валидируем корректность внесения категориальных данных на основе описания и проверяем по основным статистическим показателям для категориальных величин
df["surgery?"] = df["surgery?"].apply(validate_categorical_values, args=(1, 2))
print(df["surgery?"].value_counts().sort_index())
print(df["surgery?"].mode())
print(df["surgery?"].unique(), "\n")

df["Age"] = df["Age"].apply(validate_categorical_values, args=(1, 2))
print(df["Age"].value_counts().sort_index())
print(df["Age"].mode())
print(df["Age"].unique(), "\n")

df["temperature of extremities"] = df["temperature of extremities"].apply(
    validate_categorical_values, args=(1, 2, 3, 4)
)
print(df["temperature of extremities"].value_counts().sort_index())
print(df["temperature of extremities"].mode())
print(df["temperature of extremities"].unique(), "\n")

df["pain"] = df["pain"].apply(validate_categorical_values, args=(1, 2, 3, 4, 5))
print(df["pain"].value_counts().sort_index())
print(df["pain"].mode())
print(df["pain"].unique(), "\n")

df["outcome"] = df["outcome"].apply(validate_categorical_values, args=(1, 2, 3))
print(df["outcome"].value_counts().sort_index())
print(df["outcome"].mode())
print(df["outcome"].unique(), "\n")

df.head()  # распределение данных соответствует логике описания (например живых лошадей существенно больше, чем мертвых и взрослых больще чем жеребят), а так же мы проверили, что категориальная маркировка в точности соответствует заявленной в описании (если есть расхождение мы заменяем это значение на np.nan)

surgery?
1.0    180
2.0    118
Name: count, dtype: int64
0    1.0
Name: surgery?, dtype: float64
[ 1.  2. nan] 

Age
1    275
2     24
Name: count, dtype: int64
0    1
Name: Age, dtype: int64
[1 2] 

temperature of extremities
1.0     78
2.0     30
3.0    108
4.0     27
Name: count, dtype: int64
0    3.0
Name: temperature of extremities, dtype: float64
[nan  1.  4.  2.  3.] 

pain
1.0    38
2.0    59
3.0    67
4.0    39
5.0    41
Name: count, dtype: int64
0    3.0
Name: pain, dtype: float64
[ 3.  2. nan  4.  5.  1.] 

outcome
1.0    178
2.0     76
3.0     44
Name: count, dtype: int64
0    1.0
Name: outcome, dtype: float64
[ 3.  1.  2. nan] 



,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,1.0,1,39.2,88.0,20.0,NaN,3.0,3.0
1,2.0,1,38.3,40.0,24.0,1.0,3.0,1.0
2,1.0,2,39.1,164.0,84.0,4.0,2.0,2.0
3,2.0,1,37.3,104.0,35.0,NaN,NaN,2.0
4,2.0,1,NaN,NaN,NaN,2.0,2.0,1.0


In [704]:
# работаем с непрерывными величинами
print("Before outliers cleaning:")
print(df[["rectal temperature", "pulse", "respiratory rate"]].describe())
# описательные выводы по базовым статистикам: 1) rectal temperature - около 20% данных пропущено, основные статистические показатели не находятся в сильном разбросе, что подтверждается                                               низким значением стандартного отклонения - что говорит о притяжении данных к среднему значению
# 2) pulse - около 8% данных пропущено, очевидно, что есть выбросы (судя по max) и значение стандартного отклонения говорит о явном разбросе, поэтому для последующего анализа я бы использовал данные в межквартильном размахе q1-q3
# 3) respiratory rate - около 20% данных пропущено, есть явные выбросы по min и max. Но в описании сказано, что замеры крайне не точны и нормальный rate 8-10 а представленные даные сильно отличаются от приведенного диапазона в большую сторону (сильные превышения идут уже с q1). В связи с чем данный показатель лучше целиком исключить из анализа, как и рекомендовано в описании.


def iqr_func(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return {
        "no_outliers": series[(series >= lower_bound) & (series <= upper_bound)],
        "outliers": sorted(
            list(series[(series < lower_bound) | (series > upper_bound)])
        ),
    }


r_temp = iqr_func(df["rectal temperature"])
df["rectal temperature"] = r_temp["no_outliers"]
r_temp_outliers = pd.Series(r_temp["outliers"])
# print(r_temp_outliers)  # данные "выбросы" вероятно не являются реальными выбросами (т.е. ошибками измерения), а являются следствием необычной природы входных данных, так как в описании сказано, что возможно кратковременное повышение температуры из-за воспалительных процессов, которое приходит в норму по мере входения животного в шоковое состояние или пониженная температура в связи с длительным нахождением животного в шоковом состоянии. Значения "выбросов" я бы не стал учитывать в дальнейшем анализе принимая во внимание временный характер данных явлений.

pulse = iqr_func(df["pulse"])
df["pulse"] = pulse["no_outliers"]
pulse_outliers = pd.Series(pulse["outliers"])
# print(pulse_outliers) # это настоящие выбросы (те ошибки измерения) так как данные показатели существенно превышают физическиие возможности лошадей любых пород. Их необходимо исключить

r_rate = iqr_func(df["respiratory rate"])
df["respiratory rate"] = r_rate["no_outliers"]
r_rate_outliers = pd.Series(r_rate["outliers"])
# print(r_rate_outliers)
print("\n", "After outliers cleaning:")
print(df[["rectal temperature", "pulse", "respiratory rate"]].describe())

Before outliers cleaning:
       rectal temperature       pulse  respiratory rate
count          239.000000  275.000000        241.000000
mean            38.166527   71.934545         30.427386
std              0.733508   28.680522         17.678256
min             35.400000   30.000000          8.000000
25%             37.800000   48.000000         18.000000
50%             38.200000   64.000000         24.000000
75%             38.500000   88.000000         36.000000
max             40.800000  184.000000         96.000000

 After outliers cleaning:
       rectal temperature       pulse  respiratory rate
count          225.000000  270.000000        224.000000
mean            38.167556   70.274074         26.709821
std              0.572795   26.129625         11.486810
min             36.800000   30.000000          8.000000
25%             37.800000   48.000000         18.000000
50%             38.200000   64.000000         24.000000
75%             38.500000   88.000000         35.00

In [705]:
(df.isna().mean() * 100).round(
    2
)  # проверяем сколько % данных отсутствует в каждом столбце

surgery?                       0.33
Age                            0.00
rectal temperature            24.75
pulse                          9.70
respiratory rate              25.08
temperature of extremities    18.73
pain                          18.39
outcome                        0.33
dtype: float64

In [706]:
# task_3

# заполнение пропусков начинаем с наиболее простых и очевидных данных, чтобы в последствии было проще заполнять более сложные и менее очевидные
df = df.drop(
    columns=["respiratory rate"]
)  # удаляем весь столбец тк, большие флуктуации данных сильно отличающихся от нормы, к тому же данный показатель не надежен и польза сомнительна, как сказано в документации

df["surgery?"] = df["surgery?"].fillna(
    df["surgery?"].mode()[0]
)  # показатель выполения операции заполняем модой без других группировочных признаков, так как данных пропущено очень мало к тому же группировочные признаки не ясны причинно-следственно
df["outcome"] = df["outcome"].fillna(
    df["outcome"].mode()[0]
)  # показатель живучести лошадей так же заполняем модой, так как пропущенных данных очень мало, а живых лошадей подавляющее большинство

df["pain"] = df["pain"].fillna(df.loc[df["surgery?"] == 1, "pain"].fillna(5))
df["pain"] = df["pain"].fillna(
    df.loc[df["surgery?"] == 2, "pain"].fillna(1)
)  # в документации сказано, что категориальная величина не может быть использована как порядковая или дискретная (вероятно из-за высокой субъективности оценки), поэтому заполняю экстремальными значениями - те для лошадей не прошедших операцию ставим показатель "внимание, нет боли", что говорит о возможном сокрытие боли обезболивающими в процессе медикаментозного лечения и для лошадей прошедших операцию значение 5, те фактором операции стала сильная продолжительная боль

df["rectal temperature"] = df["rectal temperature"].fillna(
    df.groupby(["surgery?", "outcome"])["rectal temperature"].transform("mean")
)  # данный показатель наиболее статистически стабилен и предсказуем поэтому пропуски заполняем средним значением отдельно по особям прошедших операцию и нет (возможность инфекции) а так же отдельно по лошадям, которые выжили, так как погибшие особи наиболее вероятно оттянут на себя некоторые показатели повышенной температуры

df["pulse"] = df["pulse"].fillna(
    df.groupby(["Age", "pain"])["pulse"].transform("median")
)

df["temperature of extremities"] = df["temperature of extremities"].fillna(
    df.groupby(["surgery?", "outcome"])["temperature of extremities"].transform(
        lambda x: x.mode()[0]
    )
)

print((df.isna().mean() * 100).round(2))
df.info()

# Вывод - мы полностью заполнинли датафрейм значениями удалив один столбец с неинформативными данными и ни одной строки (все были заполненны по индивидуальным логическим соображениям), а так же произвели проверку на корректность внесения категорийных данных.

surgery?                      0.0
Age                           0.0
rectal temperature            0.0
pulse                         0.0
temperature of extremities    0.0
pain                          0.0
outcome                       0.0
dtype: float64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 7 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery?                    299 non-null    float64
 1   Age                         299 non-null    int64  
 2   rectal temperature          299 non-null    float64
 3   pulse                       299 non-null    float64
 4   temperature of extremities  299 non-null    float64
 5   pain                        299 non-null    float64
 6   outcome                     299 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 16.5 KB
